# 4. Model Deployment (PKL to ONNX & JSON) - Bicep Curl

**Objective:**
1. Load a specific trained model (e.g., RandomForest) from `.pkl`.
2. Load its corresponding scaler from `.pkl`.
3. Convert the scikit-learn model to ONNX format using `ModelDeployConverterUtils`.
4. Convert the scikit-learn scaler to JSON format using `ModelDeployConverterUtils`.
5. Save the ONNX model and JSON scaler to the `models/onnx/` directory.

In [ ]:
import os
import sys
import json
import logging

module_path = os.path.abspath(os.path.join('../../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from utils.model_deploy_convertor_utils import ModelDeployConverterUtils

logger = logging.getLogger('utils.model_deploy_convertor_utils')
if not logger.handlers:
    handler = logging.StreamHandler()
    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    handler.setFormatter(formatter)
    logger.addHandler(handler)
    logger.setLevel(logging.INFO)

## 4.1 Define Paths and Parameters

In [ ]:
MODEL_PKL_DIR = "../models/pkl/"
MODEL_ONNX_DIR = "../models/onnx/"
os.makedirs(MODEL_ONNX_DIR, exist_ok=True)

# --- Configuration for Left Arm ---
ARM_LEFT = "left"
MODEL_TYPE_LEFT = "NB"
SCALER_PKL_PATH_LEFT = os.path.join(MODEL_PKL_DIR, f"scaler_{ARM_LEFT}.pkl")
MODEL_PKL_PATH_LEFT = os.path.join(MODEL_PKL_DIR, f"{MODEL_TYPE_LEFT}_model_{ARM_LEFT}.pkl")
SCALER_JSON_PATH_LEFT = os.path.join(MODEL_ONNX_DIR, f"scaler_{ARM_LEFT}.json")
ONNX_MODEL_PATH_LEFT = os.path.join(MODEL_ONNX_DIR, f"bicep_model_{ARM_LEFT}.onnx") # Naming to match detection script

# --- Configuration for Right Arm ---
ARM_RIGHT = "right"
MODEL_TYPE_RIGHT = "NB"
SCALER_PKL_PATH_RIGHT = os.path.join(MODEL_PKL_DIR, f"scaler_{ARM_RIGHT}.pkl")
MODEL_PKL_PATH_RIGHT = os.path.join(MODEL_PKL_DIR, f"{MODEL_TYPE_RIGHT}_model_{ARM_RIGHT}.pkl")
SCALER_JSON_PATH_RIGHT = os.path.join(MODEL_ONNX_DIR, f"scaler_{ARM_RIGHT}.json")
ONNX_MODEL_PATH_RIGHT = os.path.join(MODEL_ONNX_DIR, f"bicep_model_{ARM_RIGHT}.onnx")

TARGET_OPSET = 12 # Common opset version

## 4.2 Conversion Function

In [ ]:
def convert_and_save(arm_name, model_type, scaler_pkl_path, model_pkl_path, scaler_json_path, onnx_model_path):
    print(f"\n--- Processing {arm_name.capitalize()} Arm ({model_type} model) ---")
    
    # Check if PKL files exist
    if not os.path.exists(scaler_pkl_path):
        print(f"ERROR: Scaler PKL file not found: {scaler_pkl_path}")
        return False
    if not os.path.exists(model_pkl_path):
        print(f"ERROR: Model PKL file not found: {model_pkl_path}")
        return False
        
    # 1. Convert Scaler to JSON
    print(f"Converting scaler {scaler_pkl_path} to {scaler_json_path}...")
    json_path = ModelDeployConverterUtils.convert_scaler_pickle_to_json(scaler_pkl_path, scaler_json_path)
    if json_path:
        print(f"Scaler successfully converted to JSON: {json_path}")
        # Verify content (optional)
        try:
            with open(json_path, 'r') as f_json:
                scaler_data = json.load(f_json)
            print(f"  JSON Scaler content (mean): {scaler_data.get('mean')[:3]}... (scale): {scaler_data.get('scale')[:3]}...")
        except Exception as e:
            print(f"  Could not verify JSON content: {e}")
    else:
        print(f"Failed to convert scaler to JSON.")
        return False # Stop if scaler conversion fails, as model might depend on it implicitly
        
    # 2. Convert Model to ONNX
    print(f"\nConverting model {model_pkl_path} to {onnx_model_path}...")
    # The convert_model_pickle_to_onnx expects model_name_no_ext
    model_name_no_ext = f"{model_type}_model_{arm_name.lower()}" # e.g., RF_model_left
    
    onnx_path = ModelDeployConverterUtils.convert_model_pickle_to_onnx(
        pickle_model_dir=MODEL_PKL_DIR, 
        onnx_model_dir=MODEL_ONNX_DIR, 
        name=model_name_no_ext, # Pass the name without .pkl extension
        target_opset=TARGET_OPSET
    )
    
    if onnx_path and os.path.exists(onnx_model_path):
        print(f"Model successfully converted to ONNX: {onnx_model_path}")
        return True
    elif onnx_path:
        print(f"Model converted by utility to: {onnx_path}, but expected {onnx_model_path}. Check naming.")
        if os.path.exists(onnx_path):
            print(f"Consider renaming {onnx_path} to {onnx_model_path} if needed for downstream tasks.")
        return False # Mark as potential issue if names don't align as expected for bicep_model_{side}.onnx
    else:
        print(f"Failed to convert model to ONNX.")
        return False

## 4.3 Perform Conversions

In [ ]:
print("Starting Left Arm Conversion...")
success_left = convert_and_save(
    arm_name=ARM_LEFT,
    model_type=MODEL_TYPE_LEFT, # This is the type of PKL model being loaded (e.g., 'RF')
    scaler_pkl_path=SCALER_PKL_PATH_LEFT,
    model_pkl_path=MODEL_PKL_PATH_LEFT, # Path to the source PKL, e.g. RF_model_left.pkl
    scaler_json_path=SCALER_JSON_PATH_LEFT,
    onnx_model_path=ONNX_MODEL_PATH_LEFT # Desired final path for the ONNX model, e.g. bicep_model_left.onnx
)
if success_left:
    print("\nRe-attempting/Verifying ONNX model conversion for Left Arm with correct naming logic...")
    source_pkl_model_base_name_left = f"{MODEL_TYPE_LEFT}_model_{ARM_LEFT}" # e.g. RF_model_left
    utility_onnx_output_path_left = os.path.join(MODEL_ONNX_DIR, f"{source_pkl_model_base_name_left}.onnx") # e.g. RF_model_left.onnx

    # Ensure the source PKL exists
    if os.path.exists(MODEL_PKL_PATH_LEFT):
        returned_onnx_path_left = ModelDeployConverterUtils.convert_model_pickle_to_onnx(
            pickle_model_dir=MODEL_PKL_DIR, 
            onnx_model_dir=MODEL_ONNX_DIR, 
            model_name_no_ext=source_pkl_model_base_name_left, # This is used to load PKL and name ONNX
            target_opset=TARGET_OPSET
        )
        if returned_onnx_path_left and os.path.exists(utility_onnx_output_path_left):
            print(f"ONNX model created by utility: {utility_onnx_output_path_left}")
            # Now, rename if necessary to match the detection script's expectation (bicep_model_left.onnx)
            if utility_onnx_output_path_left != ONNX_MODEL_PATH_LEFT:
                try:
                    os.rename(utility_onnx_output_path_left, ONNX_MODEL_PATH_LEFT)
                    print(f"Renamed ONNX model to: {ONNX_MODEL_PATH_LEFT}")
                except OSError as e:
                    print(f"Error renaming {utility_onnx_output_path_left} to {ONNX_MODEL_PATH_LEFT}: {e}")
                    print(f"Please manually rename or adjust paths.")
            else:
                print(f"ONNX model already has the target name: {ONNX_MODEL_PATH_LEFT}")
        elif returned_onnx_path_left:
            print(f"ONNX conversion returned a path {returned_onnx_path_left}, but expected file {utility_onnx_output_path_left} was not found or matched.")
        else:
            print(f"ONNX model conversion failed for Left Arm using utility.")
    else:
        print(f"Source PKL model not found for Left Arm: {MODEL_PKL_PATH_LEFT}")
else:
    print("Skipping detailed ONNX model conversion for Left Arm due to earlier failure (likely scaler).")

print("\nStarting Right Arm Conversion...")
success_right = convert_and_save(
    arm_name=ARM_RIGHT,
    model_type=MODEL_TYPE_RIGHT,
    scaler_pkl_path=SCALER_PKL_PATH_RIGHT,
    model_pkl_path=MODEL_PKL_PATH_RIGHT,
    scaler_json_path=SCALER_JSON_PATH_RIGHT,
    onnx_model_path=ONNX_MODEL_PATH_RIGHT 
)

if success_right:
    print("\nRe-attempting/Verifying ONNX model conversion for Right Arm with correct naming logic...")
    source_pkl_model_base_name_right = f"{MODEL_TYPE_RIGHT}_model_{ARM_RIGHT}"
    utility_onnx_output_path_right = os.path.join(MODEL_ONNX_DIR, f"{source_pkl_model_base_name_right}.onnx")

    if os.path.exists(MODEL_PKL_PATH_RIGHT):
        returned_onnx_path_right = ModelDeployConverterUtils.convert_model_pickle_to_onnx(
            pickle_model_dir=MODEL_PKL_DIR, 
            onnx_model_dir=MODEL_ONNX_DIR, 
            model_name_no_ext=source_pkl_model_base_name_right, 
            target_opset=TARGET_OPSET
        )
        if returned_onnx_path_right and os.path.exists(utility_onnx_output_path_right):
            print(f"ONNX model created by utility: {utility_onnx_output_path_right}")
            if utility_onnx_output_path_right != ONNX_MODEL_PATH_RIGHT:
                try:
                    os.rename(utility_onnx_output_path_right, ONNX_MODEL_PATH_RIGHT)
                    print(f"Renamed ONNX model to: {ONNX_MODEL_PATH_RIGHT}")
                except OSError as e:
                    print(f"Error renaming {utility_onnx_output_path_right} to {ONNX_MODEL_PATH_RIGHT}: {e}")
            else:
                print(f"ONNX model already has the target name: {ONNX_MODEL_PATH_RIGHT}")
        elif returned_onnx_path_right:
            print(f"ONNX conversion returned a path {returned_onnx_path_right}, but expected file {utility_onnx_output_path_right} was not found or matched.")
        else:
            print(f"ONNX model conversion failed for Right Arm using utility.")
    else:
        print(f"Source PKL model not found for Right Arm: {MODEL_PKL_PATH_RIGHT}")
else:
    print("Skipping detailed ONNX model conversion for Right Arm due to earlier failure (likely scaler).")

print("\n\nDeployment conversion process finished. Check logs for details.")
print(f"Expected ONNX models: {ONNX_MODEL_PATH_LEFT}, {ONNX_MODEL_PATH_RIGHT}")
print(f"Expected JSON scalers: {SCALER_JSON_PATH_LEFT}, {SCALER_JSON_PATH_RIGHT}")

2025-06-05 12:33:32,578 - utils.model_deploy_convertor_utils - INFO - Starting scaler to JSON conversion for 'scaler_left.pkl': ../models/pkl/scaler_left.pkl -> ../models/onnx/scaler_left.json
2025-06-05 12:33:32,580 - utils.model_deploy_convertor_utils - INFO - Successfully loaded scaler from ../models/pkl/scaler_left.pkl
2025-06-05 12:33:32,581 - utils.model_deploy_convertor_utils - INFO - Successfully converted and saved scaler parameters to ../models/onnx/scaler_left.json
2025-06-05 12:33:32,584 - utils.model_deploy_convertor_utils - INFO - Starting conversion for model 'NB_model_left': ../models/pkl/NB_model_left.pkl -> ../models/onnx/NB_model_left.onnx
2025-06-05 12:33:32,597 - utils.model_deploy_convertor_utils - INFO - Successfully loaded pickle model from ../models/pkl/NB_model_left.pkl
2025-06-05 12:33:32,598 - utils.model_deploy_convertor_utils - INFO - Model for 'NB_model_left' expects 2 input features.
2025-06-05 12:33:32,599 - utils.model_deploy_convertor_utils - INFO - A

Starting Left Arm Conversion...

--- Processing Left Arm (NB model) ---
Converting scaler ../models/pkl/scaler_left.pkl to ../models/onnx/scaler_left.json...
Scaler successfully converted to JSON: ../models/onnx/scaler_left.json
  JSON Scaler content (mean): [102.5552315619491, 12.838442030328471]... (scale): [66.81539933761593, 8.460554254478405]...

Converting model ../models/pkl/NB_model_left.pkl to ../models/onnx/bicep_model_left.onnx...
Model converted by utility to: ../models/onnx/NB_model_left.onnx, but expected ../models/onnx/bicep_model_left.onnx. Check naming.
Consider renaming ../models/onnx/NB_model_left.onnx to ../models/onnx/bicep_model_left.onnx if needed for downstream tasks.
Skipping detailed ONNX model conversion for Left Arm due to earlier failure (likely scaler).

Starting Right Arm Conversion...

--- Processing Right Arm (NB model) ---
Converting scaler ../models/pkl/scaler_right.pkl to ../models/onnx/scaler_right.json...
Scaler successfully converted to JSON: ../m

**Verification Note:**
After running, check the `exercises/bicep_curl/models/onnx/` directory for:
- `scaler_left.json`
- `scaler_right.json`
- `bicep_model_left.onnx`
- `bicep_model_right.onnx`

The `model_deploy_convertor_utils.py` uses the `model_name_no_ext` argument to both load the `.pkl` file (e.g., `MODEL_PKL_DIR/RF_model_left.pkl`) and to name the output `.onnx` file (e.g., `MODEL_ONNX_DIR/RF_model_left.onnx`). 
The code above first calls the utility which will create `RF_model_left.onnx`. Then, it renames this file to `bicep_model_left.onnx` to match the naming convention expected by the live detection script. This renaming step is crucial if the source PKL base name differs from the desired ONNX base name.